# SETUP:  Read data and make a map

We're going to read some data from the web, then use it to make a map.

The dataset is here: https://pubs.usgs.gov/dds/dds-033/USGS_3D/ssx_txt/3dstart.htm

This is the dataset we'll start with: https://pubs.usgs.gov/dds/dds-033/USGS_3D/data_set/ssxporos.dat

## Using `pandas`

It's a bit easier, once we allow for multiple separator characters (using regex) and pass in the columns:

In [1]:
columns = 'API             X            Y      SBASDP   SARDEP  SAMPDP    POR     PERM'.split()
columns

['API', 'X', 'Y', 'SBASDP', 'SARDEP', 'SAMPDP', 'POR', 'PERM']

In [2]:
import pandas as pd

url = 'https://pubs.usgs.gov/dds/dds-033/USGS_3D/data_set/ssxporos.dat'

df = pd.read_csv(url, sep='\s+', names=columns, na_values=['*'])

df.head()

,API,X,Y,SBASDP,SARDEP,SAMPDP,POR,PERM
0,4900505027,35002,-32080,30.0,-41.0,8456,4.0,NaN
1,4900505027,35002,-32080,28.0,-43.0,8458,4.0,NaN
2,4900505027,35002,-32080,26.0,-45.0,8460,4.5,NaN
3,4900505027,35002,-32080,24.0,-47.0,8462,2.4,NaN
4,4900505027,35002,-32080,22.0,-49.0,8464,5.0,NaN


What the columns mean:

```
API   ( 1:10)  American Petroleum Institute well identification number
X     (14:18)  Approximate distance in meters east of the map origin
Y     (22:27)  Approximate distance in meters north or south of the map origin
SBASDP (31:33)  Distance (in ft) of sample above the basal disconformity of
                            the Sussex "B" sandstone
SARDEP (37:41)  Distance (in ft) of sample below the base of the Ardmore bentonite
SAMPDP (45:48)  Depth (in ft) of the sample below ground surface
POR    (52:55)  Core or well-log porosity (in percent)
PERM   (59:62)  Core permeability (mD) for about 13 wells.  An * indicates no data
```

In [3]:
df.describe()

,API,X,Y,SBASDP,SARDEP,SAMPDP,POR,PERM
count,2.580000e+03,2580.000000,2580.000000,2580.000000,2580.000000,2580.000000,2580.000000,239.000000
mean,4.900524e+09,33073.703876,-25383.325581,15.667829,-54.909302,8135.743023,8.646039,10.440460
std,4.469248e+03,5987.574831,6555.046151,15.041552,14.002337,151.422423,3.843442,17.925802
min,4.900505e+09,6579.000000,-39135.000000,-40.000000,-98.000000,7774.000000,-1.000000,0.010000
25%,4.900523e+09,28500.000000,-30925.000000,4.000000,-66.000000,8032.000000,5.710000,0.425000
50%,4.900523e+09,32092.000000,-25893.000000,15.000000,-54.500000,8131.000000,8.170000,2.100000
75%,4.900524e+09,38497.000000,-19882.000000,26.000000,-44.000000,8222.000000,11.600000,12.750000
max,4.900560e+09,47061.000000,1269.000000,57.000000,-6.000000,8947.000000,21.900000,154.000000


In [4]:
len(df.API.unique())

121

In [5]:
df.groupby('API').count().X

API
4900505027    16
4900505032    28
4900520576    19
4900521053    30
4900521255    26
              ..
4900525678    18
4900527320    27
4900527466    25
4900527774    24
4900560052    25
Name: X, Length: 121, dtype: int64

## Tops

Let's read the `ssxtops.dat` file: https://pubs.usgs.gov/dds/dds-033/USGS_3D/data_set/ssxtops.dat

In [6]:
columns = 'API              X           Y           TRS         ELEV  YR    TD  ARDEP  SSXTP SSXBD  SSXUT SSXUB  SSXLT  SSXLB  SYMB  NAME'.split()

In [7]:
import pandas as pd

url = 'https://pubs.usgs.gov/dds/dds-033/USGS_3D/data_set/ssxtops.dat'

dt = pd.read_csv(url, sep='\s+', names=columns, index_col=False, na_values=['*'])

dt = dt.set_index('API')

dt.head()

,X,Y,TRS,ELEV,YR,TD,ARDEP,SSXTP,SSXBD,SSXUT,SSXUB,SSXLT,SSXLB,SYMB,NAME
API,,,,,,,,,,,,,,,
4900525150,51540,-39668,N42W7102,4886,79,9370,7501,7536.0,7536.0,NaN,NaN,NaN,NaN,22,1
4900524446,50740,-39992,N42W7103,4904,77,9476,7583,7618.0,7618.0,NaN,NaN,NaN,NaN,22,1
4900523952,49120,-40002,N42W7104,4952,75,9589,7716,7750.0,7750.0,NaN,NaN,NaN,NaN,20,4-4
4900524166,48300,-39197,N42W7104,5013,76,9688,7802,7850.0,7866.0,NaN,NaN,NaN,NaN,20,2-4
4900524791,47080,-39185,N42W7105,5007,77,9726,7840,7886.0,7917.0,NaN,NaN,NaN,NaN,22,2-5


An explanation of those columns:

```
API  (  1:10 ) American Petroleum Institute well identification number
X     ( 15:19 ) Approximate distance in meters east of the map origin
Y     ( 23:28 ) Approximate distance in meters north or south of the map origin
TRS   ( 32:39 ) Township direction (north or south) and number, range 
                 direction (west or east) and number.  Alpha format
ELEV  ( 43:46 ) Surface elevation of the borehole, in feet
YR    ( 50:51 ) Well completion year
TD    ( 55:59 ) Total drill depth from the surface, in feet
ARDEP  ( 63:66) Depth (in ft) from the surface to the Ardmore bentonite
SSXTP  ( 70:73) Depth (in ft) from the surface to the disconformity between
                  the Sussex "B" sandstone and the overlying Cody Shale.
SSXBD  ( 77:80) Depth (in ft) from the surface to the basal disconformity 
                  between the Sussex "B" sandstone and the Cody Shale.
SSXUT  ( 84:87) Depth (in ft) from the surface to the top of the upper reservoir sandstones.
SSXUB  ( 91:94) Depth (in ft) from the surface to the base of the Sussex "B" 
                   upper reservoir sandstones.
SSXLT  ( 98:101) Depth (in ft) from the surface to the top of the Sussex "B"
                   lower reservoir sandstones.
SSXLB  (105:108) Depth (in ft) from the surface to the base of the Sussex "B"
                   lower reservoir sandstones.
SYMB  (112:113) Well symbols of dry and non-productive (17), injection (15),
                   gas wells (20), oil (22), and oil and gas (29) wells
NAME  (113:132) Well name - in mixed alpha and numeric characters
```

## Pull together

We'll make one file per well.

A set of tops *more or less* in order.

Plus porosity data... which will be in the zones, but don't know which so we'll make a separate section for them.

Would be hard to read this with `pandas`.

In [8]:
df.head()

,API,X,Y,SBASDP,SARDEP,SAMPDP,POR,PERM
0,4900505027,35002,-32080,30.0,-41.0,8456,4.0,NaN
1,4900505027,35002,-32080,28.0,-43.0,8458,4.0,NaN
2,4900505027,35002,-32080,26.0,-45.0,8460,4.5,NaN
3,4900505027,35002,-32080,24.0,-47.0,8462,2.4,NaN
4,4900505027,35002,-32080,22.0,-49.0,8464,5.0,NaN


In [9]:
dt.head()

,X,Y,TRS,ELEV,YR,TD,ARDEP,SSXTP,SSXBD,SSXUT,SSXUB,SSXLT,SSXLB,SYMB,NAME
API,,,,,,,,,,,,,,,
4900525150,51540,-39668,N42W7102,4886,79,9370,7501,7536.0,7536.0,NaN,NaN,NaN,NaN,22,1
4900524446,50740,-39992,N42W7103,4904,77,9476,7583,7618.0,7618.0,NaN,NaN,NaN,NaN,22,1
4900523952,49120,-40002,N42W7104,4952,75,9589,7716,7750.0,7750.0,NaN,NaN,NaN,NaN,20,4-4
4900524166,48300,-39197,N42W7104,5013,76,9688,7802,7850.0,7866.0,NaN,NaN,NaN,NaN,20,2-4
4900524791,47080,-39185,N42W7105,5007,77,9726,7840,7886.0,7917.0,NaN,NaN,NaN,NaN,22,2-5


In [10]:
dt.columns[6:13]

Index(['ARDEP', 'SSXTP', 'SSXBD', 'SSXUT', 'SSXUB', 'SSXLT', 'SSXLB'], dtype='object')

In [11]:
import random

random.shuffle

<bound method Random.shuffle of <random.Random object at 0x559c9f6044c0>>

In [24]:
!rm ../data/sussex/*.txt

In [28]:
!rm ../data/sussex/*.tops

In [29]:
main_area = [
 4900525104,
 4900524202,
 4900523719,
 4900523357,
 4900525239,
 4900525209,
 4900523777,
 4900525135,
 4900523631,
 4900524791,
 4900523444,
 4900524816,
 4900524005,
 4900525166,
 4900523753,
 4900523857,
 4900523603,
 4900523499,
 4900522918,
 4900525133,
 4900523972,
 4900523389,
 4900523863,
 4900523827,
 4900525287,
 4900523595,
 4900524006,
 4900523520,
 4900524276,
 4900525207,
 4900525199,
 4900523683,
 4900523835,
 4900523138,
 4900523650,
 4900521053,
 4900527320,
 4900523333,
 4900525134,
 4900523570,
 4900524228,
 4900525116,
 4900527774,
 4900523649,
 4900525200,
 4900523535,
 4900523013,
 4900523478,
 4900523208,
 4900524277,
 4900523816,
 4900525210,
 4900525111,
 4900523964,
 4900523633,
]

In [30]:
fms = {
    'ARDEP': 'Ardmore',
    'SSXTP': 'Cody',
    'SSXBD': 'Niobrara',
    'SSXUT': 'Sussex Upper Top',
    'SSXUB': 'Sussex Upper Base',
    'SSXLT': 'Sussex Lower Top',
    'SSXLB': 'Sussex Lower Base',
}

meta = ['X', 'Y', 'YR', 'ELEV', 'TD']

# perc = [4900505027, 4900520576, 4900521053, 4900521255]
perc = []
force = [ 4900525104, 4900524202,]

import random
import math

count = 0

for api, group in df.groupby('API'):

    if api not in main_area:
        continue
    elif api == force[0]:
        comment, ordered, units = '#', True, 'M'
    elif api == force[1]:
        comment, ordered, units = '#', False, 'FT'
    elif api in perc:
        # Put these in the % batch.
        comment, ordered, units = '#', True, 'FT'
    else:
        comment = '#' if random.random() > 0.5 else '%'
        ordered = True if random.random() > 0.5 else False
        units = 'M' if random.random() > 0.5 else 'FT'

    prefix = 'UWI_' if comment == '#' else ''
    sep = ',' if comment == '#' else '\t'
    mult = 0.3048 if units == 'M' else 1
    xmult = 1 if units == 'M' else 1  # NOT CONVERTING COORDS
    pmult = 0.01 if units == 'M' else 1

    try:
        tops = [(mult*float(depth), top) for top, depth in zip(fms.keys(), dt.loc[api, fms.keys()])]
    except KeyError:
        continue

    pors = [(depth, por) for depth, por in zip(mult * group.SAMPDP, pmult * group.POR)]
    x, y, yr, kb, td = dt.loc[api, meta].values

    if ordered:
        random.shuffle(tops)
    else:
        tops.sort()

    text = f"{comment} LOC: {x/xmult:.2f} M,{y/xmult:.2f} M\n"
    text += f"{comment} KB: {mult*kb:.2f} {units}\n{comment} TD: {mult*td:.2f} {units}\n{comment} UNITS: {units}\n"
    text += f"{comment} Formations\n"
    text += "\n".join([f"{fms[top]}{sep}{-999.25 if math.isnan(depth) else depth:.2f}" for depth, top in tops]) + '\n'
    text += f"{comment} Porosity\n"
    text += "\n".join([f"{depth:.2f}{sep}{por:.3f}" for depth, por in pors]) + '\n'

    with open(f'../data/sussex/{prefix}{api}.tops', 'wt') as f:
        count += 1
        f.write(text)
        
print(count, "files written")

55 files written
